Initializing Google Earth Engine and importing geemap

In [1]:
import ee
import geemap
import json
import os
from geemap import geojson_to_ee
from ipyleaflet import GeoJSON
from my_gee_functions import file_address_func
from my_gee_functions import ee_list_func

In [3]:
# Authenticate if needed
ee.Authenticate()

Enter verification code:  4/1AfJohXk7lrJZlEMZG6AE4WtXKx87vRTFAXbWKVgruWs3e5Ow8rTLXaNizWE



Successfully saved authorization token.


In [2]:
#Initializing the ee project

ee.Initialize(project = 'ee-jdawsey')

https://github.com/gee-community/geemap/blob/master/examples/notebooks/07_geojson.ipynb

In [3]:
#file_dir = 'C:/Users/Justin/Desktop/mesquite/mesq_jsons' # if on desktop
file_dir = 'C:/Users/Justin-Laptop/Documents/Research/GEE_LPC/mesq_jsons' # if on laptop

listy = file_address_func(file_dir)
listy[1] # checking that each each file path was generated

'C:/Users/Justin-Laptop/Documents/Research/GEE_LPC/mesq_jsons/aa.json — aa.geojson'

In [4]:
ee_items = ee_list_func(listy)

ee_items[1] # checking that the given json was turned into an ee item

Adding jsons as ee assets

Importing each geojson for use in GEE

In [5]:
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

Loading in each polygon for data to get clipped to.

Lots of mesquite: 31, 25, 12
Grassland mix: 17,
Grassland: 18,
Farmland: 5, 30
Farm and grass: 19

In [6]:
poly_num = 25
shp = ee_items[poly_num] # list

year = ee.ImageCollection('USDA/NAIP/DOQQ').filterDate('2016-01-01', '2016-12-31').filterBounds(shp)

year = year.mosaic() # mosaicing so that becomes a single image that can be worked with
clip = year.clip(shp) # clip to the polygon bounds

visParam = {'bands' : ['N', 'R', 'G'],
           'gamma' : 1}

#Map.setCenter(shp.geometry())
Map.addLayer(clip, visParam)

Adding indices for additional possible accuracy and standardizing

SAVI (Soil Adjusted Vegetation Index):
SAVI = (1 + L) * (Bnir - Bred) / (Bnir + Bred + L)
--- L = correction factor (0 = high veg, 1 = very little veg)

ENDVI (Enhanced Normalized Difference Vegetation Index):
ENDVI = ((NIR + Green) - (2 * Blue)) / ((NIR + Green) + (2 * Blue))

In [7]:
from my_gee_functions import naip_savi_endvi
from my_gee_functions import stdrd_func

n_clip = naip_savi_endvi(clip)
standardized = stdrd_func(n_clip, shp)

Gaussian smoothing then DoG sharpening

In [8]:
from my_gee_functions import gauss_smooth_func
from my_gee_functions import dog_sharp

gauss = gauss_smooth_func(standardized)
gauss = dog_sharp(gauss)
vis_param_gauss = {'bands' : ['N', 'R', 'G'], 
               'min' : -1, 
               'max' : 2,
               'gamma' : 1}

### checking that image was smoothed
#Map.addLayer(gauss, vis_param_gauss)

https://google-earth-engine.com/Advanced-Image-Processing/Object-based-Image-Analysis/

https://gis.stackexchange.com/questions/273658/performing-object-based-image-classification-in-google-earth-engine

In [9]:
bands = ['R', 'G', 'B', 'N', 'savi', 'endvi']

seed_spacing = 5
seeds = ee.Algorithms.Image.Segmentation.seedGrid(seed_spacing, 'hex')

# Run SNIC on the regular square grid.
snic = ee.Algorithms.Image.Segmentation.SNIC(
  image = gauss,
  #size = 4, # shouldn't matter since have seed image
  compactness = 0, # spatial weighting not taken into account at 0. More compact at higher values
  connectivity = 8, # 4 or 8
  #neighborhoodSize = 256,
  seeds = seeds # the "seedGrid" given
).select(['R_mean', 'G_mean', 'B_mean', 'N_mean', 'savi_mean', 'endvi_mean', 'clusters'], ['R', 'G', 'B', 'N', 'savi', 'endvi', 'clusters'])

clusters = snic.select('clusters')
#Map.addLayer(clusters.randomVisualizer(), {}, 'clusters')
#Map.addLayer(snic, {'bands': ['R',  'G',  'B'], 'min':0, 'max':1, 'gamma': 0.8}, 'means', False)

# Compute per-cluster stdDev.
stdDev = gauss.addBands(clusters).reduceConnectedComponents(ee.Reducer.stdDev(), 'clusters', 256)
#Map.addLayer(stdDev, {'min':0, 'max':0.1}, 'StdDev', False)

# Area, Perimeter, Width and Height
area = ee.Image.pixelArea().addBands(clusters).reduceConnectedComponents(ee.Reducer.sum(), 'clusters', 256)
#Map.addLayer(area, {'min':50000, 'max': 500000}, 'Cluster Area', False)

minMax = clusters.reduceNeighborhood(ee.Reducer.minMax(), ee.Kernel.square(1))
perimeterPixels = minMax.select(0).neq(minMax.select(1)).rename('perimeter')
#Map.addLayer(perimeterPixels, {'min': 0, 'max': 1}, 'perimeterPixels')

perimeter = perimeterPixels.addBands(clusters) \
    .reduceConnectedComponents(ee.Reducer.sum(), 'clusters', 256)
#Map.addLayer(perimeter, {'min': 100, 'max': 400}, 'Perimeter size', False)

sizes = ee.Image.pixelLonLat().addBands(clusters).reduceConnectedComponents(ee.Reducer.minMax(), 'clusters', 256)
width = sizes.select('longitude_max').subtract(sizes.select('longitude_min')).rename('width')
height = sizes.select('latitude_max').subtract(sizes.select('latitude_min')).rename('height')
#Map.addLayer(width, {'min':0, 'max':0.02}, 'Cluster width', False)
#Map.addLayer(height, {'min':0, 'max':0.02}, 'Cluster height', False)

objectPropertiesImage = ee.Image.cat([
  snic.select(bands),
  stdDev,
  area,
  perimeter,
  width,
  height
]).float()

#training = objectPropertiesImage.addBands(cdl2016.select('cropland')) \
#    .updateMask(seeds) \
#    .sample(geometry, 5)
#classifier = ee.Classifier.smileRandomForest(10).train(training, 'cropland')
#Map.addLayer(objectPropertiesImage.classify(classifier), {'min':0, 'max':254}, 'Classified objects')
#Map

Segmented image training

In [12]:
max_clust = 13

In [13]:
### creating training samples for the unsupervised classification
training = objectPropertiesImage.sample(
  region = shp,
  scale = 1, #change depending on year
  numPixels = 10000
)

### the actual classification function
clusterer = ee.Clusterer.wekaXMeans(maxClusters = max_clust, # change clusters depending on imagery
                                    maxIterations = 5, 
                                    useKD = True).train(training)

result = objectPropertiesImage.cluster(clusterer)


In [15]:
obj_palette = ['#000000','#a6cee3', '#1f78b4', '#b2df8a', '#33a02c', '#fb9a99', '#e31a1c', '#fdbf6f', '#ff7f00', '#cab2d6', '#6a3d9a', '#ffff99', '#b15928']

obj_vis = {'min' : 0, 'max' : 12, 'palette' : obj_palette}

Map.addLayer(result, obj_vis)

Pixel based - Training the classification algorithm and showing the display parameters

In [51]:
### creating training samples for the unsupervised classification
pb_training = gauss.sample(
  region = shp,
  scale = 1, #change depending on year
  numPixels = 10000
)

In [52]:
### the actual classification function
pb_clusterer = ee.Clusterer.wekaXMeans(maxClusters = max_clust, # change clusters depending on imagery
                                    maxIterations = 5, 
                                    useKD = True).train(pb_training)

pixel_based_result = gauss.cluster(pb_clusterer)

#Map.addLayer(pixel_based_result)

Downloading imagery

drive_image <- ee_image_to_drive(
  image = result,
  description = "export",
  folder = "!imagery",
  region = shp,
  scale = 1,
  max = 130000000
)

ee_export_image(ee_object, filename, scale=None, crs=None, region=None, file_per_band=False)

In [53]:
# If the export has more than 1e8 pixels, set "maxPixels" higher.
task = ee.batch.Export.image.toDrive(
    image = result,
    description = f'poly{poly_num}_obj_class_size{seed_spacing}_maxClust{max_clust}',
    folder = '!imagery',
    region = shp.geometry(),
    scale = 1,
    crs = None,
    maxPixels = 130000000
)
task.start()


In [54]:
task = ee.batch.Export.image.toDrive(
    image = pixel_based_result,
    description = f'poly{poly_num}_pb_class_maxClust{max_clust}',
    folder = '!imagery',
    region = shp.geometry(),
    scale = 1,
    crs = None,
    maxPixels = 130000000
)
task.start()

In [55]:
task = ee.batch.Export.image.toDrive(
    image = standardized,
    description = f'poly{poly_num}_stdrd_ref',
    folder = '!imagery',
    region = shp.geometry(),
    scale = 1,
    crs = None,
    maxPixels = 130000000
)
task.start()